# Intent Classification

Für die Intent Clasification wird ein Feedforward Neuronales Netz impementiert und trainert mit einem Bag of Words Ansatz

## Import benötigter Bibliotheken

In [ ]:
# import of necessary libraries
import nltk 
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import datefinder
import numpy
import tflearn 
import tensorflow
import random 
import json
import pickle
import spacy
import sqlite3
import re

## Trainingsdaten vorbereiten, Implementation und Training des Neuronalen Netzes

In [ ]:
# open and load json file with patterns --> questions and tags
with open("intents.json") as file:
    data = json.load(file)

In [ ]:
# prepare training data + implement and train Neural Net
try: 
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = [] #all word in the json file
    labels = [] #all intents in the json file
    docs_x = [] #input of user
    docs_y = [] #intent for each input

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            #Stemming - Getting the root words with tokenize
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    # Neural Network versteht nur Nummern --> Bag of words
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag =  []
        wrds = [stemmer.stem(w) for w in doc]

        for w in words: 
            if w in wrds:
                bag.append(1)
            else: 
                bag.append(0)


        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    # in array für model
    training = numpy.array(training)
    output = numpy.array(output)
    
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)
        
#create, train and save neural net
tensorflow.compat.v1.reset_default_graph()
    
# Creating the Neural Network
net = tflearn.input_data(shape= [None, len(training[0])]) #input layer Neurons = numer of words in training
net = tflearn.fully_connected(net, 8) #hidden layer fully connected with 8 neuron
net = tflearn.fully_connected(net, len(output[0]), activation="softmax" ) #output layer 6 Neurons = labels
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")
